# add c_EC as one of the inputs for diffusivity and conductivity

In [1]:
import pybamm
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd   ;import numpy as np;import os;import matplotlib.pyplot as plt;import os;
from scipy.io import savemat,loadmat;from pybamm import constants,exp,tanh;import matplotlib as mpl; fs=17; # or we can set import matplotlib.pyplot as plt then say 'mpl.rc...'
for k in range(0,1):
    mpl.rcParams["axes.labelsize"] = fs
    mpl.rcParams["axes.titlesize"] = fs
    mpl.rcParams["xtick.labelsize"] =  fs
    mpl.rcParams["ytick.labelsize"] =  fs
    mpl.rcParams["legend.fontsize"] =  fs
    mpl.rcParams['font.sans-serif'] = ['Times New Roman']
    mpl.rcParams['font.serif'] = ['Times New Roman']
    mpl.rcParams['axes.unicode_minus'] = False

In [2]:
# define function:
def electrolyte_conductivity_Nyman2008_wEC(c_e,c_EC, T):
    x = c_EC / c_e
    """ index = np.piecewise(
        x, 
        [x < 0, ((x >= 0) & (x < 4.541)), x >= 4.541], 
        [lambda x : 0, lambda x : 1/4.541*x, lambda x : 1]) """
    sigma_e = (
        (c_EC <= 4541) * (x/4.541) * (
        0.1297 * (c_e / 1000) ** 3 - 2.51 * (c_e / 1000) ** 1.5 + 3.329 * (c_e / 1000)) 
        + (c_EC > 4541) * (
        0.1297 * (c_e / 1000) ** 3 - 2.51 * (c_e / 1000) ** 1.5 + 3.329 * (c_e / 1000))
    )
    return sigma_e
def electrolyte_diffusivity_Nyman2008_wEC(c_e, c_EC,T):
    x = c_EC / c_e
    """ index = np.piecewise(
        x, 
        [x < 0, ((x >= 0) & (x < 4.541)), x >= 4.541], 
        [lambda x : 0, lambda x : 1/4.541*x, lambda x : 1]) """
    D_c_e = (
        (c_EC <= 4541) * (1/4.541*x) * (
        8.794e-11 * (c_e / 1000) ** 2 - 3.972e-10 * (c_e / 1000) + 4.862e-10) 
        +  (c_EC > 4541) *  (
        8.794e-11 * (c_e / 1000) ** 2 - 3.972e-10 * (c_e / 1000) + 4.862e-10)
    )
    return D_c_e


In [3]:
V_max = 4.2;        V_min = 2.5;
Diff_SEI = 1.7e-20;    R_SEI = 2E5;   Bulk_Sol_Con = 4541.0;
Experiment_Long   = pybamm.Experiment( [ (
    f"Discharge at 1 C until {V_min} V", 
    f"Charge at 0.3 C until {V_max} V", 
    f"Hold at {V_max} V until C/5")    ] * 1 ) 
Experiment_s   = pybamm.Experiment( [ (
    f"Discharge at 1 C for 197 s", )    ] * 1 )   

In [4]:
BasicPath = 'D:/OneDrive - Imperial College London/SimDataSave/P3R4/'; 
Target  = 'a2_Diff_Cond_c_EC/'
if not os.path.exists(BasicPath + Target):
    os.mkdir(BasicPath + Target);

In [5]:
Model_ddiff = pybamm.lithium_ion.DFN(     
    options={"solvent diffusion": "EC wo refill"     } )
Model_sdiff = pybamm.lithium_ion.DFN(     
    options={"solvent diffusion": "EC w refill"     } )
ChemistryChen=pybamm.parameter_sets.Chen2020 
ChemistryChen["electrolyte"] = "lipf6_Nyman2008_rio";
Para_0=pybamm.ParameterValues(chemistry=ChemistryChen);


Para_0['EC Lithium ion cross diffusivity [m2.s-1]'] = 1.5e-9/1e2;
Para_0['Typical EC Lithium ion cross diffusivity [m2.s-1]'] =   1.5e-9/1e2
Para_0['EC transference number'] =    -1.4
Para_0['EC diffusivity in electrolyte [m2.s-1]'] =  5E-12;
""" Para_0.update(
    {'Electrolyte diffusivity [m2.s-1]':
    electrolyte_diffusivity_Nyman2008_wEC})
Para_0.update(
    {'Electrolyte conductivity [S.m-1]':
    electrolyte_conductivity_Nyman2008_wEC}) """
c_e_ddiff = Model_ddiff.variables["Electrolyte concentration [mol.m-3]"]
T_ddiff = Model_ddiff.variables["Cell temperature [K]"]
c_EC_ddiff = Model_ddiff.variables["EC concentration [mol.m-3]"]


c_e_sdiff = Model_sdiff.variables["Electrolyte concentration [mol.m-3]"]
T_sdiff = Model_sdiff.variables["Cell temperature [K]"]
c_EC_sdiff = Model_sdiff.variables["EC concentration [mol.m-3]"]


Model_ddiff.variables["c(EC) over c(Li+)"] = c_EC_ddiff / c_e_ddiff
#Model_ddiff.variables["Electrolyte conductivity [S.m-1]"] =electrolyte_conductivity_Nyman2008_wEC(c_e_ddiff,c_EC_ddiff, T_ddiff)
#Model_ddiff.variables["Electrolyte diffusivity [m2.s-1]"] =electrolyte_diffusivity_Nyman2008_wEC(c_e_ddiff,c_EC_ddiff, T_ddiff)
Model_ddiff.variables["Electrolyte conductivity [S.m-1]"] =Para_0['Electrolyte conductivity [S.m-1]'](c_e_ddiff,c_EC_ddiff, T_ddiff)
Model_ddiff.variables["Electrolyte diffusivity [m2.s-1]"] =Para_0['Electrolyte diffusivity [m2.s-1]'](c_e_ddiff,c_EC_ddiff, T_ddiff)
Model_sdiff.variables["c(EC) over c(Li+)"] = c_EC_sdiff / c_e_sdiff
#Model_sdiff.variables["Electrolyte conductivity [S.m-1]"] =electrolyte_conductivity_Nyman2008_wEC(c_e_sdiff,c_EC_sdiff, T_sdiff)
#Model_sdiff.variables["Electrolyte diffusivity [m2.s-1]"] =electrolyte_diffusivity_Nyman2008_wEC(c_e_sdiff,c_EC_sdiff, T_sdiff)
Model_sdiff.variables["Electrolyte conductivity [S.m-1]"] =Para_0['Electrolyte conductivity [S.m-1]'](c_e_sdiff,c_EC_sdiff, T_sdiff)
Model_sdiff.variables["Electrolyte diffusivity [m2.s-1]"] =Para_0['Electrolyte diffusivity [m2.s-1]'](c_e_sdiff,c_EC_sdiff, T_sdiff)

using EC wo refill for Li+
using EC w refill for Li+


In [6]:
Sim_Landes_ddiff    = pybamm.Simulation(
    Model_ddiff, experiment = Experiment_Long,
    parameter_values = Para_0,
    solver = pybamm.CasadiSolver(),)   
Sim_Landes_sdiff    = pybamm.Simulation(
    Model_sdiff, experiment = Experiment_Long,
    parameter_values = Para_0,
    solver = pybamm.CasadiSolver(),)  
Sol_Landes_ddiff    = Sim_Landes_ddiff.solve();
Sol_Landes_sdiff    = Sim_Landes_sdiff.solve();

In [7]:
Sol_Landes_ddiff["Electrolyte conductivity [S.m-1]"].entries[:,-1] 

array([0.94700724, 0.9470191 , 0.94704275, 0.94707807, 0.94712485,
       0.94718279, 0.9472515 , 0.94733049, 0.94741912, 0.9475166 ,
       0.94762203, 0.94773429, 0.94785212, 0.94797409, 0.94809862,
       0.94822401, 0.94834844, 0.94847007, 0.94858705, 0.94869758,
       0.94875234, 0.94875793, 0.94876351, 0.94876909, 0.94877466,
       0.94878022, 0.94878576, 0.94879129, 0.9487968 , 0.9488023 ,
       0.94880777, 0.94881323, 0.94881866, 0.94882406, 0.94882944,
       0.9488348 , 0.94884012, 0.94884541, 0.94885067, 0.9488559 ,
       0.94888543, 0.94893891, 0.94899077, 0.94904039, 0.94908732,
       0.94913124, 0.94917192, 0.94920926, 0.9492432 , 0.94927377,
       0.94930104, 0.9493251 , 0.94934606, 0.94936405, 0.9493792 ,
       0.94939163, 0.94940145, 0.94940874, 0.94941356, 0.94941597])

In [8]:
font = {'family' : 'Times New Roman',
        #'weight' : 'bold',
        'size'   : 18}
mpl.rc('font', **font)

In [9]:
label = ["without refill","with refill",] 
output_variables3 = [
    "Terminal voltage [V]",   
    #"Current [A]",
    "EC concentration [mol.m-3]",
    "Electrolyte concentration [mol.m-3]",
    "c(EC) over c(Li+)",
    #"Loss of capacity to SEI [A.h]",
    "Li+ flux [mol.m-2.s-1]",
    "EC flux [mol.m-2.s-1]",
    "Electrolyte conductivity [S.m-1]",
    "Electrolyte diffusivity [m2.s-1]",
]
quick_plot = pybamm.QuickPlot([
    Sol_Landes_ddiff,Sol_Landes_sdiff
    ], output_variables3,label,
    variable_limits='fixed',time_unit='hours',n_rows=2,
    figsize = (16,6)) #     spatial_unit='mm',
quick_plot.dynamic_plot();
#quick_plot.create_gif(
#    number_of_images=10, duration=2,output_filename=BasicPath + Target+"fixed_add c_EC as linear.gif")

interactive(children=(FloatSlider(value=0.0, description='t', max=4.1620043951311505, step=0.04162004395131150…